<div style="text-align: right"><b> Ana Valentina López Chacón & Isabella Martínez Martínez & Sara Julieth Zuleta Quevedo. </b></div>

# **Proyecto 3: Detección de Odio y Generando Alegría**

Este proyecto tiene como objetivo que aprendan a utilizar los transformers en aplicaciones de la vida real, usando el ecosistema de https://huggingface.co/ (librerías de transformers, datasets, tokenizers, etc) y PyTorch. \

Utilizarán como base un transformer derivado de BERT llamado [RoBERTuito](https://huggingface.co/pysentimiento/robertuito-base-uncased) que ha sido entrenado (en español) sobre más de 500 millones de tweets.  El paper de dicho modelo lo pueden encontrar en este [link](https://arxiv.org/abs/2111.09453).

### **Parte I. Detección de Odio**

El discurso del Odio, lamentablemente se esta volviendo muy frecuente, e inclusive se pueden programar bots para promoverlo. En esta parte utilizarán un modelo de HF llamado [robertuito-hate-speech](https://huggingface.co/pysentimiento/robertuito-hate-speech). La idea es que utilicen la API de Twitter que usaron en el primer proyecto para recopilar una gran cantidad de tweets (durante la última semana/día(s) en **Colombia**) y detecten cuales de esos tweets están siendo "odiosos". Dicho modelo produce tres etiquetas (es multi-label): HS-Hate Speach; TR - targeted to specific individual y; AG - Aggressive. Una probabilidad relativamente alta en cualquier de esas categorías indica que es un tweet de odio.

Este problema no requiere ningún entrenamiento, es sólo usar el modelo de transformer mencionado. 

¿Qué usuarios en los últimos días estan frecuentemente usando un discurso de odio? ¿Serán Bots?

Generé un pipeline (una función, script, etc.) que pueda variar el tiempo de recopilación y así poder determinar diferentes momentos en los que se esta hablando de esta forma. 

Para esta parte les puede ser útil la documentación de HF o el capítulo 2 y 3 del libro de [Natural Language Processing with Transformers](https://github.com/nlp-with-transformers/notebooks)

### **Parte II. Contrarestar el Odio**

Como seguramente se esta generando odio a través de tweets  el objetivo de esta parte es crear un bot que genere un discurso de alegría.

RoBERTuito también tiene un modelo para detección de emociones: [robertuito-emotion-analysis](https://huggingface.co/pysentimiento/robertuito-emotion-analysis). La idea es que usen una estrategia similar a la de la parte 1 para identificar que tweets en español son los que producen la etiqueta "joy", durante la última semana (no necesariamente limitado a Colombia). Guarde estos tweets porque serviran para realizar un generador de alegría. 

Ahora, deberán usar un modelo de transformer basado en GPT2 (por ejemplo: datificate/gpt2-small-spanish, PlanTL-GOB-ES/gpt2-large-bne, DeepESP/gpt2-spanish) y realizar un fine-tuning en la generación del texto sobre el dataset de alegría conseguido anteriormente (use muy pocos epochs, 2 o 3). Evalúe los resultados sobre el conjunto de validación usando una métrica para modelo de lenguaje (por ejemplo Perplexity). 

Una vez obtenido el modelo de generación llevelo a producción con una de las dos formas siguientes: (1) realice una interfaz para la generación del texto o (2) use la API de twitter para realizarlo de forma automática.

Para éste punto recomiendo revisar lo siguiente:

[1] Capítulo 5 del libro de [Natural Language Processing with Transformers](https://github.com/nlp-with-transformers/notebooks)

[2] [Este notebook sobre generación de texto](https://github.com/huggingface/blog/blob/main/notebooks/02_how_to_generate.ipynb)

[2] [Este notebook sobre fine-tune un modelo de generación de texto](https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb)


# **Implementación**

**DISCLAIMER:** Correr el notebook desde la sección **Parte I** con el fin de optimizar recursos y tiempo.

### **Obtención de datos**


Se realiza el proceso de obtención de datos mediante la API, sin embargo, esta recolección se ejecutó previamente y ya se obtuvo el dataset. Por esto, es posible omitir correr esta sección.

In [ ]:
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time

In [ ]:
os.environ["TOKEN"] = "AAAAAAAAAAAAAAAAAAAAAOo3ZQEAAAAAttwki7TuUfCUeGuwxKei4oABaO8%3DTI93KR3JpuwT4p5kDw8ScQHl9dnjwLp9T7nnvaLHTDRjNK6O3K"

In [ ]:
def auth():
    return os.getenv("TOKEN")

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, start_date, end_date, max_results=10):
    search_url = "https://api.twitter.com/2/tweets/search/recent"

    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)


def connect_to_endpoint(url, headers, params, next_token=None):
    params['next_token'] = next_token
    response = requests.request("GET", url, headers=headers, params=params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
def append_to_csv(json_response, fileName):
    counter = 0
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    for tweet in json_response['data']:
        author_id = tweet['author_id']
        created_at = dateutil.parser.parse(tweet['created_at'])
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "
        tweet_id = tweet['id']
        lang = tweet['lang']
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']
        source = tweet['source']
        text = tweet['text']
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        csvWriter.writerow(res)
        counter += 1

    csvFile.close()

    print("# of Tweets added from this response: ", counter) 

Con el fin de recolectar la información pertinente de los tweets de Colombia en la última semana, se agregaron ciertos parámetros que permiten realizar esta tarea. Se tienen las listas `start_list` y `end_list` las cuales definen las fechas a considerar y el parámetro `max_count` que define el límite de la cantidad de tweets a recolectar.

In [ ]:
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "Colombia lang:es"
start_list =    ['2022-05-01T00:00:00.000Z', # Si desea cambiar las fechas de inicio, modifique esta lista
                 '2022-05-02T00:00:00.000Z',
                 '2022-05-03T00:00:00.000Z',
                 '2022-05-04T00:00:00.000Z',
                 '2022-05-05T00:00:00.000Z']

end_list =      ['2022-05-02T00:00:00.000Z', # Si desea cambiar las fechas de finalización, modifique esta lista
                 '2022-05-03T00:00:00.000Z',
                 '2022-05-04T00:00:00.000Z',
                 '2022-05-05T00:00:00.000Z',
                 '2022-05-06T00:00:00.000Z']
max_results = 100

total_tweets = 0

csvFile = open("more_data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    count = 0 
    max_count = 20000 
    flag = True
    next_token = None
    
    while flag:
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "more_data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(1)                
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "more_data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(1)
            
            flag = False
            next_token = None
        time.sleep(1)
print("Total number of results: ", total_tweets)


Independiente del valor asignado a las variables mencionadas anteriormente, estos datos se guardan bajo el nombre de `more_data.csv` y para importarlos se debe emplear el nombre correspondiente.

### **Parte I: Detección de Odio.**

Se descargan e importan las librerías necesarias para el desarrollo de esta sección, como `pysentimiento` y `transformers`.



In [ ]:
!pip install transformers

In [ ]:
!pip install pysentimiento

In [ ]:
import pandas as pd 
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import re
from IPython.display import display, HTML

from datasets import Dataset, load_dataset, ClassLabel
from pysentimiento.preprocessing import preprocess_tweet
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import AutoConfig
from transformers import  AutoModelForCausalLM

import gradio as gr

A partir de los resultados obtenidos en la sección `Obtención de Datos`, se leen los tweets recolectados como un `dataset` y se especifican los parámetros o atributos que se tiene para cada tweet.

In [ ]:
from datasets import load_dataset
headers = ['author_id', 'created_at', 'geo', 'tweet_id', 'lang', 'like_count', 'quote_count', 'reply_count', 'retweet_count', 'source', 'text']
dataset = load_dataset('csv', data_files='more_data.csv', column_names=headers)

Using custom data configuration default-9e98cc883947a637


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9e98cc883947a637/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset = dataset['train']
dataset

Dataset({
    features: ['author_id', 'created_at', 'geo', 'tweet_id', 'lang', 'like_count', 'quote_count', 'reply_count', 'retweet_count', 'source', 'text'],
    num_rows: 78904
})

Con el fin de optimizar el procedimiento a realizar se desprecian los atributos que no aportan información relevante para el desarrollo del ejercicio.

In [ ]:
dataset = dataset.remove_columns(['created_at', 'geo', 'tweet_id', 'lang', 'like_count', 'quote_count', 'reply_count', 'retweet_count', 'source'])

Para detectar palabras de odio en español y así generar una clasificación binaria de la información previamente recolectada, se tomó como referencia el modelo predeterminado `robertuito-hate-speech` de `pysentimiento`, el cual es un clasificador múltiple que localiza palabras de odio. A partir de este se obtiene el *tokenizer* junto con el *modelo* a emplear.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-hate-speech")
model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-hate-speech")

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/838k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/956 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

Para obtener un resultado coherente es necesario realizar la limpieza o preprocesamiento de los datos correctamente, para esto se emplea la función `preprocess_tweet`. Además, con el propósito de optimizar recursos computacionales se aplica el *tokenizer* por secciones o *batches*.

In [ ]:
ds_pre = dataset.map(lambda x: {'text':preprocess_tweet(x['text'])})

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

ds_pre = ds_pre.map(tokenize,batched=True, batch_size = None)

0ex [00:00, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Luego, se visualizan los atributos resultantes de aplicar el *tokenizar* a cada uno de los datos.

In [ ]:
ds_pre.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'author_id': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'text': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

Dado que la función `.map()` retorna un iterable de tipo `map` se modifica este formato y se traduce a la librería `Torch`, manteniendo los mismos atributos.

In [ ]:
ds_pre.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])

Empleando la función `TrainingArguments()` se obtienen los parámetros de entrenamiento necesarios para lograr el funcionamiento adecuado del *Trainer*, con un *batch_size* de 64.

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer",
                   per_device_eval_batch_size=64)    

Posteriormente, se entrena el modelo con los parámetros determinados anteriormente.

In [ ]:
trainer = Trainer(
    model= model.to(device), 
    args=training_args,
    train_dataset=ds_pre,
    eval_dataset=ds_pre,
)

Finalmente se realiza la predicción con los tweets recolectados y el modelo ya entrenado, con el fin de identificar los tweets de odio.

In [ ]:
preds = trainer.predict(ds_pre)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: author_id, text. If author_id, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 78904
  Batch size = 64


Luego, a partir de la predicción se muestra el vector resultante para cada tweet donde se cuantifica la medida que determina si es un tweet de odio.

In [ ]:
logits = preds.predictions
logits

array([[-0.06830226, -2.1926413 , -0.8142784 ],
       [-1.7932749 , -5.0288033 , -2.7180498 ],
       [ 3.0683837 ,  1.260223  ,  1.6388124 ],
       ...,
       [-3.33056   , -4.6524396 , -4.3306866 ],
       [-2.8498938 , -4.286405  , -3.323695  ],
       [-2.8595455 , -5.2348504 , -3.6442533 ]], dtype=float32)

Ahora bien, con el propósito de facilitar el entendimiento de la predicción realizada se utiliza la función `.softmax()` para modificar la escala de los vectores resultantes y que se puedan interpretar como probabilidades, es decir, su rango será de 0 a 1 y su suma será igual a 1.

In [ ]:
logits_probs = F.softmax(torch.tensor(logits), dim = 1)

78904

Se convierte el *dataset* al tipo `DataFrame` de `Pandas` con el fin de obtener mejor visualización y facilitar la adición de las etiquetas.

In [ ]:
df = pd.DataFrame(dataset)
df.head()

Ahora, se genera un vector de etiquetas binarias, donde 1 representa un discurso de odio y 0 no. Esta clasificación se determinó empleando las probabilidades previamente encontradas y tomando un umbral de 0.6.

In [ ]:
hate_speech = np.zeros(len(df))
hate_speech[logits_probs[:,0] > 0.6] = 1

Luego, se agrega este vector como otra columna del `DataFrame`. Así, es posible visualizar la clasificación para los primeros 5 tweets de forma más agradable.

In [ ]:
df['hate_speech'] = hate_speech

In [ ]:
df.head()

,author_id,text,hate_speech
0,108861705,RT @BlancoGustavo10: El Cerdo de Colombia @Iva...,1.0
1,1043351352255160320,@JuanCar99077589 @elojodiestro @intiasprilla Y...,1.0
2,3636778473,RT @Vahiaaa: Piedad Córdoba simplemente es una...,1.0
3,878061735483240449,RT @perez_otro: Entre los compradores de biene...,0.0
4,1366056822352863232,RT @jbedoyalima: Es mi deseo pensar que @IBeta...,1.0


-----------------------------------------

Finalmente, se guarda el resultado del proceso realizado en un `dataset` bajo el nombre de `tweets_hate.csv`.

In [ ]:
]df.to_csv("tweets_hate.csv", index = False)

Dado que Twitter es una red social generalmente usada como medio de protesta, es muy probable que los datos recolectados sufran cambios volátiles con respecto a las fechas establecidas, por ejemplo, si se cambia la ubicación a *Estados Unidos* en la última semana, se encontraran demasiados comentarios de odio con respecto a la penalización del aborto que se quiere imponer en todo el país. Además, para identificar si el origen de estos datos es un bot se deberían considerar los atributos referentes al tiempo de publicación de tweets de un mismo usuario y comparar la diferencia entre ellos, para que, si se publican prácticamente en el mismo segundo, seguramente son producto de un bot.


## **Parte 2. Contrarestar el Odio**

Tomando como referencia el modelo predeterminado `robertuito-hate-speech` de `pysentimiento`, es posible realizar detección de emociones en general, no solo de odio. Dada esta característica es posible separar los tweets que sean felices para así generar texto alegre. Al igual que en la sección anterior, se obtiene el *tokenizer* y el *modelo*.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-emotion-analysis")
model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-emotion-analysis")

Realizando un proceso análogo al anterior, se realiza el preprocesamiento de los datos con la función `preprocess_tweet` y se aplica el *tokenizer* por secciones o *batches*.


In [ ]:
ds_pre = dataset.map(lambda x: {'text':preprocess_tweet(x['text'])})

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

ds_pre = ds_pre.map(tokenize,batched=True, batch_size = None)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9e98cc883947a637/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-a5f44fa03e2aba29.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Se muestran los atributos resultantes de aplicar el *tokenizer*.

In [ ]:
ds_pre.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'author_id': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'text': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

Al igual que en la sección anterior, se modifica el formato iterable de `map` y se traduce a la librería `Torch`, manteniendo los mismos atributos.

In [ ]:
ds_pre.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])

Posteriormente, se determinan los argumentos necesarios para entrenar el modelo de forma análoga al caso visto previamente.

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer",
                   per_device_eval_batch_size=64)    

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Y finalmente, se entrena el modelo con los parámetros previamente establecidos.

In [ ]:
trainer = Trainer(
    model= model.to(device), 
    args=training_args,
    train_dataset=ds_pre,
    eval_dataset=ds_pre,
)

Dado que se desea identificar emociones en general, no solo el discurso de odio, pues la clasificación deja de ser binaria y se convierte en múltiple. Para lograr esto, se toma `robertuito-emotion-analysis` también de `pysentimiento`.

In [ ]:
config = AutoConfig.from_pretrained("pysentimiento/robertuito-emotion-analysis")

Luego, se realiza el proceso de predicción naturalmente.

In [ ]:
preds = trainer.predict(ds_pre)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: author_id, text. If author_id, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 78904
  Batch size = 64


Ahora, para determinar las etiquetas se toma el argumento máximo, es decir, la emoción con mayor probabilidad y se etiqueta correspondientemente.

In [ ]:
pred_labels = [config.id2label[x] for x in preds.predictions.argmax(1)]

In [ ]:
len(pred_labels)

78904

Luego, se genera un `DataFrame` y se agrega la columna con la etiqueta del sentimiento que corresponde.

In [ ]:
df = pd.DataFrame(dataset)
df['sentiment'] = pred_labels
df.head()

,author_id,text,sentiment
0,108861705,RT @BlancoGustavo10: El Cerdo de Colombia @Iva...,anger
1,1043351352255160320,@JuanCar99077589 @elojodiestro @intiasprilla Y...,anger
2,3636778473,RT @Vahiaaa: Piedad Córdoba simplemente es una...,anger
3,878061735483240449,RT @perez_otro: Entre los compradores de biene...,others
4,1366056822352863232,RT @jbedoyalima: Es mi deseo pensar que @IBeta...,others


Dado que nos interesa generar discurso de alegría, se crea un `DataFrame` únicamente con los tweets clasificados bajo la etiqueta de `joy`.

In [ ]:
df_joy = df.loc[df['sentiment'] == 'joy']
df_joy.head()

,author_id,text,sentiment
43,470692220,RT @DiesolarteDiego: soy un viajero MTB y aman...,joy
45,1391742913860747265,RT @ClaroMusicaCO: ✨ ARMY ✨: Ustedes vienen pi...,joy
54,193359374,@IvanDuque @Europarl_ES Gracias presidente.Ha ...,joy
55,1400566191387811852,“He vuelto a inscribir mi nombre para competir...,joy
69,1222430880855183360,RT @InfoFreestyle2: ¡FMS Colombia 🇨🇴!\r\n\r\nM...,joy


In [ ]:
df_joy.to_csv('tweets_felices.csv', index=False)

In [ ]:
len(df_joy)

6455

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:510: FutureWarning: HfApi.login: This method is deprecated in favor of `set_access_token` and will be removed in v0.7.
  FutureWarning,


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Con el propósito de generar texto coherente, se debe realizar un preprocesamiento de cada tweet para suprimir factores que pueden afectar el proceso de generación de texto, empleando la función `preprocess_tweet`.

In [ ]:
df_joy['clean_text'] = df_joy['text'].apply(lambda x: preprocess_tweet(x))

In [ ]:
df_joy.head()

,author_id,text,sentiment,clean_text
43,470692220,RT @DiesolarteDiego: soy un viajero MTB y aman...,joy,RT @usuario: soy un viajero MTB y amante dela ...
45,1391742913860747265,RT @ClaroMusicaCO: ✨ ARMY ✨: Ustedes vienen pi...,joy,RT @usuario: emoji chispas emoji ARMY emoji...
54,193359374,@IvanDuque @Europarl_ES Gracias presidente.Ha ...,joy,@usuario @usuario Gracias presidente.Ha puesto...
55,1400566191387811852,“He vuelto a inscribir mi nombre para competir...,joy,"""He vuelto a inscribir mi nombre para competir..."
69,1222430880855183360,RT @InfoFreestyle2: ¡FMS Colombia 🇨🇴!\r\n\r\nM...,joy,RT @usuario: ¡FMS Colombia emoji bandera colo...


A partir de los tweets felices, se realiza la partición de los datos en entrenamiento y prueba. Además, se guardan en su respectivo path bajo un archivo `.txt`.

In [ ]:
def build_text_files(tweets, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for text in tweets:
        text = text.strip()
        text = re.sub(r"\s", " ", text)
        data += text + "\n"
    f.write(data)

train, test = train_test_split(df_joy["clean_text"], test_size = 0.2)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 5164
Test dataset length: 1291


Tomando el modelo `gpt2-small-spanish` de `transformers` para obtener el *tokenizer* y poder cargar los datos como *dataset* con los path previamente establecidos.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish", use_fast = True)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
datasets = load_dataset("text", data_files={"train": train_path, "validation": test_path})

Using custom data configuration default-1b364ac4fe5948fa


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-1b364ac4fe5948fa/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Con el fin de ejemplificar la calidad de los datos que se emplearan para generar texto, se genera la siguiente función que muestra diversos elementos de los tweets felices con el estilo de un `DataFrame`.

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,text
0,GOAL! Millonarios in Colombia Liga BetPlay Jaguares de Córdoba 0-1 Millonarios
1,RT @usuario: Con orgullo y respeto entonamos las notas del himno nacional de la República de Colombia url
2,RT @usuario: De Colombia pal mundo!!! El primero de muchos!!! emoji balón de fútbol emoji !!! Inmenso!!! url
3,"RT @usuario: ¡Sin duda alguna, Colombia hoy es Pacto Histórico!"
4,@usuario PETRO PRESIDENTE DE COLOMBIA EN PRIMERA VUELTA emoji corazón azul emoji emoji corazón amarillo emoji emoji arcoíris emoji emoji amanecer emoji congreso Pacto histórico emoji bíceps flexionado emoji 2022 emoji jarras de cerveza brindando emoji emoji jarra de cerveza emoji SALUD
5,"RT @usuario: ¡Si cambia Cali, cambia Colombia! Colombia es consciente que puede transformar su historia eligiendo un gobierno alternat"
6,RT @usuario: A los empresarios les digo: ¡los necesito; Colombia los necesita! Todos juntos. Desde el que genera un empleo hasta el q
7,RT @usuario: Familia y Amigos... Feliz Día.! Evento Importante Este Fin de Semana... Colombia En El Foco del Mundo.! emoji globo terráqueo con meridianos emoji emoji pulgar hacia arriba emoji emoji cien puntos emoji emoji manos en oración emoji emoji bandera colombia emoji emoji bandera unión europea emoji
8,"RT @usuario: emoji chincheta emoji Los soldados de colombia son el gran baluarte de nuestra institución, ellos con valentía y coraje entregan hasta su pr"
9,"RT @usuario: Orgullo de Colombia emoji bandera colombia emoji ver en el mall más grande del mundo, @usuario, y en la parte central de la tienda oficial del @usuario"


Se define el *tokenizer* y se aplica a todos los datos.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
block_size = 128

Se define la siguiente función para concatenar los tweets y separarlos o agregar un padding tal que el dato no supere una longitud máxima y estandarice el tamaño de los datos.

In [ ]:
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

Posteriormente, se aplica la función *tokenizer* a los datos modificados.

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Finalmente, se genera el modelo que permitirá generar texto.

In [ ]:
model = AutoModelForCausalLM.from_pretrained("datificate/gpt2-small-spanish")

Se determinan los parámetros de entrenamiento necesarios para el desarrollo del ejercicio.

In [ ]:
training_args = TrainingArguments(
    "datificate/gpt2-small-spanish",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    logging_steps = 1
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Luego, se define el *trainer* con los parámetros definidos y el modelo mencionado.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

Cloning https://huggingface.co/BelisaDi/gpt2-small-spanish into local empty directory.


Así, se procede a entrenar el modelo.

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1579
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 594


Epoch,Training Loss,Validation Loss
1,3.900900,3.149565
2,2.922200,2.867778
3,2.093400,2.796194


***** Running Evaluation *****
  Num examples = 401
  Batch size = 8
***** Running Evaluation *****
  Num examples = 401
  Batch size = 8
Saving model checkpoint to datificate/gpt2-small-spanish/checkpoint-500
Configuration saved in datificate/gpt2-small-spanish/checkpoint-500/config.json
Model weights saved in datificate/gpt2-small-spanish/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 401
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=594, training_loss=3.253022905552026, metrics={'train_runtime': 213.4746, 'train_samples_per_second': 22.19, 'train_steps_per_second': 2.783, 'total_flos': 309435088896000.0, 'train_loss': 3.253022905552026, 'epoch': 3.0})

Para visualizar el resultado obtenido, se realiza una evaluación del modelo mediante el parámetro `Perplexity` que mide que tan bien una distribución de probabilidad está prediciendo una muestra.

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 401
  Batch size = 8


Perplexity: 16.38


Luego, se da una demostración a partir del texto *Estoy feliz* para ver como el modelo genera una frase completa.

In [ ]:
# set top_k to 50
text = "Estoy feliz"
input_ids = tokenizer.encode(text, return_tensors='pt').to(device)

sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Estoy feliz de escuchar a millones de personas que le digo, agradecen. Por una parte vamos a darlo a la campaña para un cambio en Colombia y así saber que quieren una Colombia que nos garantice a sí mismo, una Colombia con libertad,


Es posible notar como se obtiene un desempeño satisfactorio, dado que la mayoría de temas tratados en Twitter en Colombia tienen que ver con política, es lógico afirmar que este resultado tiene sentido. Además, considerando que, de los aproximados 78.000 tweets recolectados inicialmente, solo 8.000 son clasificados como felices.

A continuación, tenemos la interfaz gráfica.

In [ ]:
def model_generation(text):
  input_ids = tokenizer.encode(text, return_tensors='pt').to(device)

  sample_output = model.generate(
      input_ids, 
      do_sample=True, 
      max_length=50, 
      top_k=50
  )

  
  return tokenizer.decode(sample_output[0], skip_special_tokens=True)

In [ ]:
iface = gr.Interface(fn=model_generation, inputs="text", outputs="text")
iface.launch()

## **Conclusiones**

Para finalizar, es posible destacar las siguientes conclusiones:

- Los modelos que ya han pasado por un proceso previo de entrenamiento de `HuggingFace` representan una herramienta poderosa con el fin de analizar sentimientos, especialmente para identificar sentimientos de odio. Esto se debe a que de por sí ya se encuentran en un estado óptimo, pues tienen diversa variedad de parámetros y fueron entrenados con una cantidad considerable de datos, lo cual otorga una mejora de gran magnitud. En particular en la primera sección, es posible destacar como identifica ciertas palabras de odio o intenciones de los usuarios de una forma bastante buena, esto tiene sentido ya que es conocimiento general que Twitter es normalmente utilizado como una red social de sátira, protesta y/o burla.

- Con respecto al desarrollo de modelos con el propósito de realizar tareas bastante especificas, el proceso de *fine-tuning* juega un papel fundamental debido al uso de pesos que son el resultado de un proceso de entrenamiento previo, pero para un propósito mucho más general pero que aún guarde cierta relación con la tarea deseada inicialmente. Esto se evidencia en la segunda sección donde a partir de un modelo capaz de identificar sentimientos, es posible generar textos donde predomine la felicidad.

- Además, tomando como referencia la generación de texto se debe tener un criterio inteligente para seleccionar la siguiente palabra en la frase, dado que escoger solo la de mayor probabilidad a ser una palabra feliz resultaría en muchas repeticiones, redundancias o simplemente oraciones sin sentido. Por ejemplo, se puede considerar algoritmos de sampling o beaming. Nuevamente, para evitar este tipo de inconsistencias las funciones de entrenamiento y construcción de modelos de `HuggingFace`, al ser un *framework* de alto nivel facilitan los procesos y cálculos necesarios para generar un resultado satisfactorio.

<div style="text-align: right"><b> Ana Valentina López Chacón & Isabella Martínez Martínez & Sara Julieth Zuleta Quevedo. </b></div>